# Modeling

---


### 1. I import the necessary libraries


In [85]:
import modin.pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

### 2. I import the data set to model


In [86]:
df_to_model_steam=pd.read_csv('/home/mkm/programin/Steam_recommendation_model/Datasets/Data_Processed/Model_Price_1.csv')


### 3. First, when preparing the data, leave all null values in the Metascore column, for the machine learning algorithm to impute, so in this section I will follow these steps.

- Preprocessing
- Treatment of columns with lists
- Divide the dataset into 2, df_train and df_test
- Train a linear regression model
- Impute the null values of the MetaScore column in our dataset.


In [89]:
df_to_model_steam

,Unnamed: 0,publisher,genres,app_name,release_date,tags,specs,price,early_access,id,developer,sentiment,metascore,year
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"Strategy, Action, Indie, Casual, Simulation",['Single-player'],4.99,False,761140.0,Kotoshiro,Neutral,NaN,2018.0
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"Free to Play, Strategy, Indie, RPG, Card Game,...","['Single-player', 'Multi-player', 'Online Mult...",4.99,False,643980.0,Secret Level SRL,Mostly Positive,NaN,2018.0
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"Free to Play, Simulation, Sports, Casual, Indi...","['Single-player', 'Multi-player', 'Online Mult...",0.00,False,670290.0,Poolians.com,Mostly Positive,NaN,2017.0
3,5,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"Action, Adventure, Simulation, FPS, Shooter, T...","['Single-player', 'Steam Achievements']",3.99,False,772540.0,Trickjump Games Ltd,Mixed,NaN,2018.0
4,7,Poppermost Productions,"['Free to Play', 'Indie', 'Simulation', 'Sports']",SNOW - All Access Pro Pass,2018-01-04,"Free to Play, Indie, Simulation, Sports","['Single-player', 'Multi-player', 'Online Mult...",18.99,False,774277.0,Poppermost Productions,Neutral,NaN,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24076,32129,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018-01-04,"Action, Indie, Casual, Violent, Adventure","['Single-player', 'Steam Achievements', 'Steam...",1.99,False,745400.0,Bidoniera Games,2 user reviews,NaN,2018.0
24077,32130,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"Strategy, Indie, Casual, Simulation","['Single-player', 'Steam Achievements']",1.99,False,773640.0,"Nikita ""Ghost_RUS""",Neutral,NaN,2018.0
24078,32131,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,"Strategy, Indie, Casual","['Single-player', 'Steam Achievements', 'Steam...",4.99,False,733530.0,Sacada,Neutral,NaN,2018.0
24079,32132,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"Indie, Simulation, Racing","['Single-player', 'Steam Achievements', 'Steam...",1.99,False,610660.0,Laush Dmitriy Sergeevich,Neutral,NaN,2018.0


In [ ]:
# Definir los valores permitidos para cada categoría
allowed_genres = ['Action', 'Adventure', 'Racing', 'Simulation', 'Strategy']
allowed_specs = ['Multi-player', 'Co-op', 'Cross-Platform Multiplayer', 'Downloadable Content']
allowed_tags = ['Simulation', 'Indie', 'Action', 'Adventure', 'Funny', 'Open World', 'First-Person', 'Sandbox', 'Free to Play']

# Filtrar las columnas 'genres', 'specs' y 'tags'
df_filtered = df_to_model_steam.copy()
df_filtered['genres'] = df_filtered['genres'].apply(lambda x: [val for val in x if val in allowed_genres])
df_filtered['specs'] = df_filtered['specs'].apply(lambda x: [val for val in x if val in allowed_specs])
df_filtered['tags'] = df_filtered['tags'].apply(lambda x: [val for val in x if val in allowed_tags])

# Convertir las listas en valores individuales utilizando 'explode'
df_filtered = df_filtered.explode('genres')
df_filtered = df_filtered.explode('specs')
df_filtered = df_filtered.explode('tags')

# Filtrar y reorganizar las columnas que se mantendrán
columns_to_keep = ['price', 'metascore', 'year', 'early_access_true', 'early_access_false', 'genres', 'specs', 'tags']
df_filtered = df_filtered[columns_to_keep]

# Mostrar el DataFrame resultante
df_filtered.head()



2023-08-08 04:15:58,200	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::_apply_list_of_funcs() (pid=6337, ip=192.168.1.7)
  File "/home/mkm/.local/lib/python3.11/site-packages/modin/core/execution/ray/implementations/pandas_on_ray/partitioning/partition.py", line 453, in _apply_list_of_funcs
    partition = func(partition, *args, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mkm/.local/lib/python3.11/site-packages/modin/core/dataframe/algebra/map.py", line 48, in <lambda>
    lambda x: function(x, *args, **kwargs), *call_args, **call_kwds
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mkm/.local/lib/python3.11/site-packages/pandas/core/frame.py", line 9640, in applymap
    return self.apply(infer).__finalize__(self, "applymap")
           ^^^^^^^^^^^^^^^^^
  File "/home/mkm/.local/lib/python3.11/site-packages/pandas/core/frame.py", line 9555, in apply
    return op.apply().__finalize__(self, method="apply

RayTaskError(TypeError): [36mray::_apply_func()[39m (pid=6337, ip=192.168.1.7)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::deploy_ray_func()[39m (pid=6337, ip=192.168.1.7)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::_apply_func()[39m (pid=6336, ip=192.168.1.7)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: [36mray::_apply_list_of_funcs()[39m (pid=6337, ip=192.168.1.7)
  File "/home/mkm/.local/lib/python3.11/site-packages/modin/core/execution/ray/implementations/pandas_on_ray/partitioning/partition.py", line 453, in _apply_list_of_funcs
    partition = func(partition, *args, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mkm/.local/lib/python3.11/site-packages/modin/core/dataframe/algebra/map.py", line 48, in <lambda>
    lambda x: function(x, *args, **kwargs), 

2023-08-08 04:16:04,757	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::_apply_func() (pid=6336, ip=192.168.1.7)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RayTaskError: ray::_apply_list_of_funcs() (pid=6336, ip=192.168.1.7)
  File "/home/mkm/.local/lib/python3.11/site-packages/modin/core/execution/ray/implementations/pandas_on_ray/partitioning/partition.py", line 453, in _apply_list_of_funcs
    partition = func(partition, *args, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mkm/.local/lib/python3.11/site-packages/modin/core/dataframe/algebra/map.py", line 48, in <lambda>
    lambda x: function(x, *args, **kwargs), *call_args, **call_kwds
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mkm/.local/lib/python3.11/site-packages/pandas/core/frame.py", line 9640, in applymap
    return self.apply(infer).__finalize__(self, "applymap")
           ^^^^^^^^^^^^^^^^^
 

In [87]:
# # Seleccionar las columnas deseadas en df_to_model_steam1
# columns_to_select = ['genres', 'tags', 'specs', 'price', 'metascore', 'year', 'early_access', 'sentiment']
# df_to_model_steam1 = df_to_model_steam[columns_to_select]

# # Filtrar las filas donde 'metascore' no es NaN
# df_to_model_steam1 = df_to_model_steam1.dropna(subset=['metascore'])

# # Lista de columnas categóricas
# categorical_columns = ['genres', 'tags', 'specs', 'sentiment']

# # Obtener las columnas dummies utilizando el prefijo predeterminado
# df_to_model_steam1 = pd.get_dummies(df_to_model_steam1, columns=categorical_columns)

# # Obtener las columnas dummies para 'early_access' y cambiar los nombres
# df_dummies = pd.get_dummies(df_to_model_steam1['early_access'], columns='early_access')
# df_dummies.rename(columns={'False': 'early_access_false', 'True': 'early_access_true'}, inplace=True)

# # Descartar la columna 'early_access' original
# df_to_model_steam1.drop(columns='early_access', inplace=True)

# # Concatenar los DataFrames utilizando merge
# df_to_model_steam1 = pd.merge(df_to_model_steam1, df_dummies, left_index=True, right_index=True)
# # Convertir la columna 'year' a tipo de datos entero (int)
# df_to_model_steam1['year'] = pd.to_numeric(df_to_model_steam1['year'], errors='coerce').astype('Int64')
# df_to_model_steam1

In [88]:
# # Lista de columnas para renombrar con el prefijo 'genres_'
# genres_columns = ['Action', 'Adventure', 'Racing', 'Simulation', 'Strategy']
# genres_prefix = 'genres_'
# genres_columns_rename = [f'{genres_prefix}{col}' for col in genres_columns]

# # Lista de columnas para renombrar con el prefijo 'specs_'
# specs_columns = ['Multi-player', 'Co-op', 'Cross-Platform Multiplayer', 'Downloadable Content']
# specs_prefix = 'specs_'
# specs_columns_rename = [f'{specs_prefix}{col}' for col in specs_columns]

# # Lista de columnas que comienzan con 'sentiment'
# sentiment_columns = [col for col in df_to_model_steam1.columns if col.startswith('sentiment')]

# # Lista de columnas a mantener
# columns_to_keep = ['price', 'metascore', 'year', 'early_access_true', 'early_access_false']

# # Filtrar y renombrar las columnas
# filtered_columns = genres_columns_rename + specs_columns_rename + sentiment_columns + columns_to_keep
# df_filtered = df_to_model_steam1[filtered_columns]

# # Crear un nuevo DataFrame con las columnas filtradas y renombradas
# df_filtered_and_renamed = df_filtered.copy()